#  The participation of Facebook group members with Social network analysis


##### Student name: Chia Wei Tu                         Student ID:300289967

In [6]:
from selenium import webdriver
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import datetime

## Use selenium to open the browser, go to Facebook, enter the account password, and click login.

In [7]:
def get_htmltext(username, password):
    
    profile = webdriver.FirefoxProfile()
    profile.set_preference("dom.webnotifications.enabled", False)     
    profile.update_preferences()     
    driver = webdriver.Firefox(firefox_profile=profile)    
    driver.get("http://upload.facebook.com")
    time.sleep(3)
    driver.find_element_by_id("email").send_keys(username)
    driver.find_element_by_id("pass").send_keys(password)
    driver.find_element_by_id("u_0_b").click()
    time.sleep(3)
    driver.get('https://upload.facebook.com/groups/237012520046062')
    time.sleep(3)
       
   #Scroll the window and click to hide the message
    for i in range(12):
        y = 4000 * (i + 1)
        driver.execute_script(f"window.scrollTo(0, {y})")
        time.sleep(2)
        
        
    #Then let selenium open all the views of X messages, OOO has replied XX, and so on, one by one.
    #Some buttons only appear after they are clicked for the first time, so you need to repeat two rounds. 
    def ClickForMore():
        hrefBtns = driver.find_elements_by_tag_name('a')    
        for btn in hrefBtns:
            try:
                s = btn.get_attribute('data-testid')
            except:
                continue
            if s == 'UFI2CommentsPagerRenderer/pager_depth_1' or s == 'UFI2CommentsPagerRenderer/pager_depth_0':
                try:
                    btn.click()
                    time.sleep(1)
                except:
                    continue
                
        ClickForMore()
        ClickForMore()      


        htmltext = driver.page_source
        driver.close()
    
        return htmltext

## Parse the source code
To obtain the source code of the website and analyze it, I first use beautifulsoup to select the area of each post and message, and then compare the crawling time and member ID with the regex string.

In [5]:
def parse_htmltext(htmltext, start_date, end_date):
   
    post_persons = []
    comment_persons = []
    good_urllist = []
    ustart_date = start_date.timestamp()
    uend_date = end_date.timestamp()
    soup = BeautifulSoup(htmltext, 'lxml')
    body = soup.find('body')
    posts = body.select('div[data-pagelet="GroupFeed"]')[0]
    feed_articles = posts.select('div[role="feed"]')[0].select('div[role="article"]')
    other_articles = posts.select('div[role="article"]')
    articles = feed_articles + other_articles
    
    for article in articles:
        if article.has_attr('id'):
            try:
                post_person = re.findall('title="(.{2,20})"><div class=', str(article))[0]
            except:
                continue
            post_time = int(re.findall('data-utime="(.*?)"', str(article))[0])        
            if post_time >= ustart_date and post_time <= uend_date:                
                post_persons.append(post_person)
            try:
                good_urllist.append(re.findall('"(/ufi/reaction/profile/browser/\?.*?)"', str(article))[0])
            except:
                pass
    
        elif article.has_attr('data-testid'):            
            comment_person = re.findall('directed_target_id.*?href=".*?">(.*?)</a>', str(article))[0]  
            comment_time = int(re.findall('data-utime="(.*?)"', str(article))[0])
            if comment_time >= ustart_date and post_time <= uend_date:                    
                comment_persons.append(comment_person)                    
                try:
                    good_urllist.append(re.findall('"(/ufi/reaction/profile/browser/\?.*?)"', str(article))[0])
                except:
                    pass
    
    return post_persons, comment_persons, good_urllist

## Get the emoji
must log in to Facebook to access the symbolic links. The content of each page is very uniform and simple, so it is directly parsed.

In [8]:
def parse_good_urllist(username, password,urllist):
  
    
    output = []

    profile = webdriver.FirefoxProfile()
    profile.set_preference("dom.webnotifications.enabled", False)  # Finally, turned off webnotifications...
    profile.update_preferences()
    driver = webdriver.Firefox(firefox_profile=profile)
    driver.get("http://upload.facebook.com")
    time.sleep(3)
    driver.find_element_by_id("email").send_keys(username)
    driver.find_element_by_id("pass").send_keys(password)
    driver.find_element_by_id("u_0_b").click()
    time.sleep(3)

    for url in urllist:
        driver.get('http://upload.facebook.com/' + url)
        htmltext = driver.page_source
        soup = BeautifulSoup(htmltext, 'html.parser')
        for raw_text in soup.select('li[class="_5i_q"]'):
            output += re.findall(re.compile('aria-label="(.*?)" class="_s'),str(raw_text))            

    driver.close()
    return output

## Organize data and output the data to CSV File
Convert the post_persons, comment_persons, and emoji_persons produced earlier into counts, and then package the results into excel files for output.

In [9]:
def tidy_up_data(post_persons, comment_persons, emoji_persons):
    
    all_persons = list(set(post_persons+comment_persons+emoji_persons))
    post_times = []
    comment_times = []
    emoji_times = []
    
    for p in all_persons:
        post_times.append(post_persons.count(p))
        comment_times.append(comment_persons.count(p))
        emoji_times.append(emoji_persons.count(p))
    
    return pd.DataFrame(dict(PersonID=all_persons, Posts=post_times, Comments=comment_times, Emoji=emoji_times))



In [10]:
if __name__ == '__main__':
    username = 'CTU89967@gmail.com'
    password = '00000000000'

    htmltext = get_htmltext(username, password)
    post_persons, comment_persons, good_urllist = parse_htmltext(htmltext, datetime.datetime(2020,11,1), datetime.datetime(2020,12,1))
    emoji_persons = parse_good_urllist(username, password, good_urllist)
    df = tidy_up_data(post_persons, comment_persons, emoji_persons)
    df.to_excel('group_activity.xlsx', index=False)

